In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

# 1. Идеи и ход решения

Для итоговой модели я использовал три представления данных: исходное представление в виде sparse матрицы, те же данные после PCA на 500 компонент (число компонент выбрано случайно) и матрица нормированных по строке корреляций тестового датасета с обобщенными портретами классов построенным по обучающей выборке. (считается корреляция строки тестового датасета с средним значением строки в обучающей выборке для всех примеров, где присутствует тот или иной тег).

### Модели:

- SGD с функцией потерь 'modified huber' на исходном представлении
- MultinomialBayes из коробки на исходном представлении
- RF на 500 деревьев на матрице корреляций с портретами классов
- LinearRegression из коробки для предсказания количества тегов для каждого документа обучаемая на PCA представлении данных

Итоговый ответ собирался из взвешенных предсказаний SGD + MultinomianBayes + RF + нормированные корреляции
{'MNB': 9.7144514654701197e-17,
 'RF': 0.64999999999999991,
 'SGD': 0.050000000000000003,
 'Correlations': 0.29999999999999999}, тут видно, что MNB не играет, но на кросс валидации был вариант весов в котором MNB имел реальный вес, а модель проигрывала в тысячных, так что возможно MNB имеет смысл. 
 
###  Кроссвалидация и настройка параметров
Кроссвалидация на трех фолдах написана руками. 
RF учится на данных полученных полученных как результат другой модели(корреляций), так что внутри каждого из 3-х фолдов кроссвалидации происходит разбиение еще на 5 фолдов и по 4-м считаются значения корреляций для 5-ой, потом из этого собирается таблица корреляций для всего внешнего фолда и на нем учится RF. Портреты считаются тоже по всему внешнему фолду. 

Решение устроенно так, чтобы один раз посчитать таблицы корреляций, предсказания RF, SGD и MNB для трех фолдов, а потом подобрать весовые коэффициенты между моделями. 

--я пробовал добавить еще одно рабиение на фолды внутри каждого из этих трех, чтобы обучить линейную метамодель на всех предсказаниях, но результат улучшить не вышло. 

Помимо этого обучается линейная регрессия чтобы предсказывать количество тегов, и результат предсказания округляется  так, чтобы получить лучший результат на кросс валидации. В таком варианте получается, что общее количество предсказанных тегов приблизительно совпадает с реальным. В качестве ответа выбирается такое количество элементов с максимальными вероятностями, которое предсказывает регрессия
--в целом этот шаг дает не очень большой прирост, кажется, несколько тысячных. Тут я пробовал много подходов. Изначально просто выбирал 1 тег с максимальной вероятностью, так же пробовал устанавливать порог по вероятности, пробовал везде выбирать один тег по максимальной вероятности, а второй добавлять, только если он выше порога, но изменения от всех таких подходов не велики.


### Что не сработало\не успел

- Предсказывать как отдельные классы парные, тройные,... шестерные теги, тогда всего получается около 600 классов. Я пробовал тут только RF + корреляции с не очень честной кросс валидацией просто по отдельному сету, там подход себя показал неудачно и я его оставил. С другой стороны 6 тегов не встречаются сами по себе, а бывают только в паре с другим, так что возможно сюда надо было копать дальше.
- Предсказывать первый тег, потом перекидывать его в признак и предсказывать второй. Вроде не сработало, но это было в 6 утра перед дедлайном, так что я не уверен, а сейчас нет времени проверить.
- PCA на 98 компонент - ничего хорошего
- LogReg и SVM на исходном сете - SGD с huber loss лучше
- Попытаться корреляции как-то калибровать в вероятности - не успел
- Вместо просто суммирования для обобщенных портретов строить суммы квадратов, это увеличивает вес значимых слов относительно остальных - кажется это дает небольшое улучение, но я не уверен. 
- добавлял куда-то какие-то сборные фичи, не помогало, забил

### P.S. 
Я не запускал все в этом ноутбуке это долго, просто подчистил и структурировал тот в котором решал, так что сходу что-то может не взлететь

# Готовим данные

In [2]:
from scipy import sparse

def read_data_from_file(filename, shape):
    values = list()
    rows = list()
    cols = list()
    
    header = True
    for line in open(filename):
        if header:
            header = False
            continue
        row, col, value = [x for x in line.strip().split(',')]
        row, col = int(row), int(col)
        value = float(value)
        row -= 1
        col -= 1
        values.append(value)
        rows.append(row)
        cols.append(col)
        
    return sparse.csr_matrix((values, (rows, cols)), shape=shape)

In [3]:
X_train = read_data_from_file('X_train.csv', (15000, 30000)).astype(float)
X_test = read_data_from_file('X_test.csv', (15000, 30000)).astype(float)
print X_train.shape, X_test.shape

(15000, 30000) (15000, 30000)


In [5]:
def read_labels_from_file(filename, shape):
    labels = np.zeros(shape).astype(int)

    header = True
    for line in open(filename):
        if header:
            header = False
            continue
        row, indeces = line.strip().split(',')
        row = int(row) - 1
        indeces = [int(x) - 1 for x in indeces.split()]
        labels[row, indeces] = 1
    
    return labels

In [6]:
y_train = read_labels_from_file('y_train.csv', (15000, 98))
print y_train.shape

(15000, 98)


In [7]:
def write_labels_to_file(labels, filename):
    outfile = open(filename, 'w')
    print >> outfile, "Id,Labels"
    for i, line in enumerate(labels):
        elements = [str(x) for x in list(np.nonzero(line)[0] + 1)]
        
        print >> outfile, "%d,%s" % (i + 1, ' '.join(elements))

# Выделяем часть датасета для кросс валидации/обучения метаалгоритмов/подбора параметров

In [32]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.grid_search import GridSearchCV

In [33]:
from sklearn.cross_validation import train_test_split

X_train_sampled, X_cv, y_train_sampled, Y_cv = train_test_split(X_train, y_train, test_size=0.30, random_state=0)
print X_train_sampled.shape
print y_train_sampled.shape

(10500, 30000)
(10500, 98)


# PCA 500

In [401]:
from sklearn.decomposition import TruncatedSVD

In [531]:
svd = TruncatedSVD(n_components=500, random_state=239)
svd.fit(X_train) 

TruncatedSVD(algorithm='randomized', n_components=500, n_iter=5,
       random_state=239, tol=0.0)

In [534]:
X_test.shape

(15000, 500)

In [ ]:
X_train_PCA = svd.transform(X_train)
X_test = svd.transform(X_test)

In [ ]:
X_test_PCA = svd.transform(X_test)

In [404]:
X_train_PCA.shape

(15000, 500)

# Попробуем выделить пары категорий

Реально выделять пары, тройки и прочее не понадобится, но от этой части решения останется представление данных в виде уникальных идентификаторов классов. Список из 599 списков, каждый из которых содержит одно из возможных сочетаний тегов, первые 92 элемента это варианты с одним тегом, потом около сотни с двумя тегами итд до 7 тегов. Больше 7 в учебном сете нет. Соответственно тут функции чтобы отлавливать уникальные наборы тегов и перекодировать Y из стандартного представление в такое и наоборот.

Важный результат этого куска - список all_nplexes - набор уникальных сочетаний, он понадобится потом.

In [11]:
def catchNplex(y, n_components = 2):
    y_nplexes = y[y.sum(1) == n_components]
    
    nplexes = list()
    
    for nplex in y_nplexes:
        #print type(nplex)
        if list(nplex) in nplexes:
            pass
        else:
            nplexes.append(list(nplex))
    
    return nplexes

In [12]:
all_nplexes = []
    
for n in range(0, 7):
    all_nplexes += catchNplex(y_train, n)

In [13]:
len(all_nplexes)

599

In [14]:
def fromYtoNplex(y_to_transform, all_nplexes):
    y_transformed = np.zeros((y_to_transform.shape[0], len(all_nplexes)))
    
    for i in range(y_to_transform.shape[0]):
        nplex_num = np.where(np.all(all_nplexes == y_to_transform[i], axis=1))[0][0]
        
        y_transformed[i][nplex_num] = 1
    
    return y_transformed

In [15]:
def fromNplextoY(y_to_transform, all_nplexes):
    y = []
    empty = [0 for i in range(98)]
    for i in range(y_to_transform.shape[0]):
        #print np.where(y_to_transform[i] == 1)
        if sum(y_to_transform[i]) != 0:
            y.append(all_nplexes[np.where(y_to_transform[i] == 1)[0][0]])
        else:
            y.append(empty)
        
    return np.array(y)

In [136]:
y_train_trainsformed = fromYtoNplex(y_train, all_nplexes)

In [49]:
y = fromNplextoY(y_train_trainsformed, all_nplexes)

# Модели

In [111]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, make_scorer
from sklearn.grid_search import GridSearchCV

In [63]:
from sklearn.linear_model import LogisticRegression

class LRClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, n_jobs=-1, t = 0.1, nplexes = all_nplexes):
        self.C = C
        self.n_jobs = n_jobs
        self.nplexes = nplexes
    
    def fit(self, X, y):
        #y_transformed = fromYtoNplex(y, self.nplexes)
        self.clf = OneVsRestClassifier(LogisticRegression(C = self.C)
                                       , self.n_jobs)
        self.clf.fit(X, y)
    
    def predict(self, X):
        #y = fromNplextoY(self.clf.predict(X), self.nplexes)
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [19]:
class SVMClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, kernel = 'linear', n_jobs=-1, nplexes = all_nplexes):
        self.C = C
        self.kernel = kernel
        self.n_jobs = n_jobs
        self.nplexes = nplexes
    def fit(self, X, y):
        y_transformed = fromYtoNplex(y, self.nplexes)
        self.clf = OneVsRestClassifier(SVC(C=self.C, kernel = self.kernel, probability=True, cache_size = 10000), self.n_jobs)
        self.clf.fit(X, y_transformed)
    
    def predict(self, X):
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [112]:
from sklearn.ensemble import RandomForestClassifier

class RFClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100, n_jobs=-1, nplexes = all_nplexes):
        self.n_estimators=n_estimators
        self.n_jobs = n_jobs
        self.nplexes = nplexes
    def fit(self, X, y):
        #y_transformed = fromYtoNplex(y, self.nplexes)
        self.clf = OneVsRestClassifier(RandomForestClassifier(self.n_estimators, n_jobs = self.n_jobs)
                                       , self.n_jobs)
        self.clf.fit(X, y)
    
    def predict(self, X):
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X)


In [114]:
from sklearn.linear_model import SGDClassifier

class SGDMulticlassClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, n_jobs=-1):
        self.C = C
        self.n_jobs = n_jobs
    
    def fit(self, X, y):
        self.clf = OneVsRestClassifier(SGDClassifier(loss='modified_huber')
                                       , self.n_jobs)
        self.clf.fit(X, y)
    
    def predict(self, X):
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

####  Пара вспомогательных функций для подготовки ответа

In [21]:
from IPython.display import Audio
sound_file = './call.wav'

ALLERT = Audio(url=sound_file, autoplay=True)

In [22]:
ALLERT

In [16]:
def postprocessWithT(y_test, t):
    y_fin = np.zeros(y_test.shape) 
    for i in range(y_test.shape[0]):
        for j in range(y_test.shape[1]):
            if y_test[i][j] >= t:
                y_fin[i][j] = 1
    return y_fin

In [186]:
from sklearn.linear_model import LinearRegression

LinReg = LinearRegression()
LinReg.fit(X_train_sampled_PCA, y_len_train_sampled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [356]:
def produceOutByLen(prop, lens):
    ans = np.zeros(prop.shape)
    print ans.shape
    for i in range(len(lens)):
        for j in range(lens[i]): 
            ans[i][np.argsort(-prop[i])[j]] = 1
    return ans     
    

# Создадим портреты классов

Вот кусочек про портреты. 

createPortraits(  ) создает портреты по списку вроде all_nplexes и обучающей выборке

predictSimple(  ) считает корреляции для тестового сета

portaitsCycle(  ) создает портреты, потом считает корреляции, потом нормирует и еще и дает оценку результата, если предсказывать просто по корреляциям тег с максимальной корреляцией

In [17]:
def createPortraits(nplexes_set, X, y, plot = False):
    portraits = []

    for i in range(nplexes_set.shape[0]):
        mask = np.all(nplexes_set[i] == y, axis =1)

        if any(mask):
            try:
                raw_masked = (np.array(X[mask].sum(0))[0])/sum(np.array(X[mask].sum(0))[0])
            except:
                print X.shape
                print mask
            portraits.append(list(raw_masked))
        else:
            portraits.append(list(np.zeros(X.shape[1])))
           
        if plot:
            plt.plot(raw_masked)

            #plt.plot(np.histogram(np.array(raw_masked)[0], bins=50)[1][1:],
            #    np.histogram(np.array(raw_masked)[0], bins=50)[0])

            #plt.plot(X_train_sampled[mask].sum(0))

            plt.show()
        
    return np.array(portraits)

In [19]:
def predictSimple(X_to_pred, portrs, rmse = False):
    y_pred = []

    for i in range(X_to_pred.shape[0]):
        cu = np.nan_to_num(X_to_pred[i].toarray()/X_to_pred[i].sum())

        corrs = []
        for p in portrs:
            if rmse:
                corrs.append(mean_squared_error(np.nan_to_num(p.reshape(1, -1)), cu))
            else:
                corrs.append(np.corrcoef(p, cu)[0][1])

        y_pred.append(corrs)
    
    return y_pred

In [20]:
def to(y_single, nplexes):
    ans = np.zeros((len(y_single), len(nplexes)))
    for i in range(len(y_single)):
        ans[i][y_single[i]] = 1
    return ans

In [21]:
def portraitsCycle(train, train_label, test, test_label, nplex_set, plot = False):
    portraits = createPortraits(nplex_set, train, train_label, plot)
    simple_pred = np.array(predictSimple(test, portraits))
    simple_pred = np.nan_to_num(simple_pred)
    simple_ans = fromNplextoY(to(simple_pred.argmax(1), nplex_set), nplex_set)
    print f1_score(test_label, simple_ans, average='samples')
    
    return simple_pred

In [ ]:
_ = portraitsCycle(X_train_sampled, y_train_sampled, X_cv, Y_cv, np.array(all_nplexes)[np.array(all_nplexes).sum(1) == 1], False)

In [352]:
_ = portraitsCycle(X_train_sampled, y_train_sampled, X_cv, Y_cv, np.array(all_nplexes)[np.array(all_nplexes).sum(1) <= 2], False)

0.437637037037


###  Тут по датасету создаем матрицу корреляций, разбиением на несколько фолдов

In [135]:
def addSimplePredFeatures(data, labels, nplex_set = np.array(all_nplexes)[np.array(all_nplexes).sum(1) == 1],
                                                                          n_folds = 5):
    folds = []
    fold_size = data.shape[0] / n_folds
     
    for i in range(n_folds):
        fold = [i*fold_size, (i+1)*fold_size]
        folds.append(fold)
    folds[n_folds - 1][1] = data.shape[0]
    
    all_new_features = None
    
    for i, j in folds:
        train_data = sparse.vstack((data[0:i],data[j:]))
        #train_data = np.delete(data, np.s_[i : j], axis = 0)
        train_labels = np.delete(labels, np.s_[i : j], axis = 0)
        data_to_predict = data[i:j]
        predict_labels = labels[i:j]
        
        #print train_data.shape
        new_features = portraitsCycle(train_data, train_labels, data_to_predict, predict_labels, nplex_set)

        if (all_new_features is None):
            all_new_features = new_features
        else:

            all_new_features = np.vstack((all_new_features, new_features))
        
    
    return all_new_features

Дальше пробуем что из этого получится на кроссвалидации по выделенному изначально датасету( это не нужно сейчас, но тут какой-то порядок оценок для алгоритмов получен) 

In [ ]:
new_features = addSimplePredFeatures(X_train_sampled, y_train_sampled, np.array(all_nplexes)[np.array(all_nplexes).sum(1) == 1], 5)

In [29]:
new_features = np.nan_to_num(new_features)
new_features_normed = (new_features / new_features.sum(1).reshape(-1,1))
new_features_normed = np.nan_to_num(new_features_normed)

In [53]:
new_features_RFclassifier = RFClassifier(n_jobs = 6)
new_features_RFclassifier.fit(new_features_normed, y_train_sampled)

In [31]:
new_features_cv = predictSimple(X_cv, portraits)

In [32]:
new_features_cv = np.nan_to_num(new_features_cv)
new_features_cv_normed = (new_features_cv / new_features_cv.sum(1).reshape(-1,1))
new_features_cv_normed = np.nan_to_num(new_features_cv_normed)

In [183]:
RF_proba = new_features_RFclassifier.predict_proba(new_features_cv_normed)

In [116]:
def nplexProbToClassProb(y, nplexes):
    #!! only works for 1-plexes
    ans = np.zeros((y.shape[0], 98))
    for i in range(y.shape[0]):
        for j in range(len(nplexes)):
            j_ind = np.where(nplexes[j] == 1)[0][0]
            ans[i][j_ind] = y[i][j]
    return ans

In [117]:
def ansFromProba(y):
    ans = np.zeros(y.shape)
    ans[np.arange(y.shape[0]), y.argmax(1)] = 1
    return ans

In [83]:
new_features_proba= nplexProbToClassProb(new_features_cv_normed, np.array(all_nplexes)[np.array(all_nplexes).sum(1) == 1])

 Вот кое-какие результаты. Сначала лес на матрице корреляций, потом просто максимум по корреляциям

In [134]:
print f1_score(Y_cv, ansFromProba(RF_proba), average='samples')

0.567133333333


In [135]:
print f1_score(Y_cv, ansFromProba(new_features_proba), average='samples')

0.519407407407


In [357]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor



In [ ]:
lens = np.around(LinReg.predict(X_cv_PCA) + 0.25)

#  Пробуем подобрать коэффициенты на RF \ corr \ SGD \ MNB

Собственно самая главная функция которая делает разбиение на три фолда для кроссвалидации. По каждым двум фолдам считаются корреляции (разбиением еще на 5 фолдов) потом учатся модели, и дается 4 предсказания (от каждой модели) для 3-его внешнего фолда. 

In [138]:
def AllFolds(data, labels, nplexes, n_folds = 3) :
    folds = []
    fold_size = data.shape[0] / n_folds
     
    for i in range(n_folds):
        fold = [i*fold_size, (i+1)*fold_size]
        folds.append(fold)
    
    folds[n_folds - 1][1] = data.shape[0]
    
    folds_probas = []
    
    for i, j in folds:
        probas = dict()
        
        train_data = sparse.vstack((data[0:i],data[j:]))
        train_labels = np.delete(labels, np.s_[i : j], axis = 0)
        cv_data = data[i:j]
        cv_labels = labels[i:j]

        new_features = addSimplePredFeatures(train_data, train_labels, nplexes, 5)
        new_features = np.nan_to_num(new_features)
        new_features_normed = (new_features / new_features.sum(1).reshape(-1,1))
        new_features_normed = np.nan_to_num(new_features_normed)
        
        print 'teaching RF ......'
        new_features_RFclassifier = RFClassifier(n_estimators = 100, n_jobs = 6)
        new_features_RFclassifier.fit(new_features_normed, train_labels)
        
        print 'RF done...... \n portraits.....'
        portraits = createPortraits(nplexes, train_data, train_labels, False)
        print 'done......'
        cv_features = predictSimple(cv_data, portraits)
        
        cv_features = np.nan_to_num(cv_features)
        cv_features_normed = (cv_features / cv_features.sum(1).reshape(-1,1))
        cv_features_normed = np.nan_to_num(cv_features_normed)
        
        cv_features_proba = nplexProbToClassProb(cv_features_normed, nplexes)
        
        RF_proba = new_features_RFclassifier.predict_proba(cv_features_normed)
        
        SGD = SGDMulticlassClassifier()
        
        SGD.fit(train_data, train_labels)
        SGD_proba = SGD.predict_proba(cv_data)
        
        MNB = MultinomialNB()
        train_data_split, train_label_split = splitData(train_data, train_labels)
        MNB.fit(train_data_split, train_label_split.argmax(1))
        
        MNB_labels = np.unique(sorted(train_labels.argmax(1)))
        MNB_proba = updateMNBproba(MNB_labels, np.arange(98), MNB.predict_proba(cv_data))
        
        print 'All probas calculated....\n'
        probas['features_normed'] = cv_features_normed
        probas['features_proba'] = cv_features_proba
        probas['RF_proba'] = RF_proba
        probas['SGD_proba'] = SGD_proba
        probas['MNB_proba'] = MNB_proba
        
        folds_probas.append(probas)
    
    return folds_probas
        

In [214]:
f_probas = AllFolds(X_train, y_train, np.array(all_nplexes)[np.array(all_nplexes).sum(1) == 1])

0.39645
0.482726190476
0.47815
0.463566666667
0.4361
teaching RF ......
RF done...... 
 portraits.....
done......
(5000, 98)
All probas calculated....

0.432833333333
0.475576190476
0.453816666667
0.452316666667
0.438983333333
teaching RF ......
RF done...... 
 portraits.....
done......
(5000, 98)
All probas calculated....

0.428783333333
0.487942857143
0.465833333333
0.45045952381
0.443833333333
teaching RF ......
RF done...... 
 portraits.....
done......
(5000, 98)
All probas calculated....



Сохраняю данные

In [215]:
for i in range(3):

    to_save = np.zeros((5000, 1))

    for key in f_probas[i]:
        if key != 'features_normed':
            to_save = np.hstack((to_save, f_probas[i][key]))
    
    pd.DataFrame(to_save).to_csv('fin_data_' + str(i)+'.csv')
    

Функция генерит словарик весов в заданных диапазонах, чтобы подобрать оптимальное соотношение моделей

In [193]:
def genParams(step= 0.05, start = [0.4,0.3,0], end = [0.8,0.8,1]):
    params_list = []
    params = dict()
    for i1 in np.arange(start[0],end[0] + 0.001,step):
        for i2 in np.arange(start[1],min(1 - i1, end[1]) + 0.001,step):
            for i3 in np.arange(start[2],min(1 - i1 - i2, end[2]) + 0.001,step):
                params = dict()
                params['RF_proba'] = i1
                params['features_proba'] = i2
                params['SGD_proba'] = i3
                params['MNB_proba'] = 1 - i1 - i2 - i3
                params_list.append(params)
                
    return params_list

In [194]:
params_list = genParams()

####  Дальше вариации с выводом ответа (по трешхолду, по длине, по одному максимуму). В качестве решения на кагл принят вариант с предсказанием одного тега и предсказанием по длине

In [199]:
s = 0
counter = 0
for params in params_list[81:82]:
    s = 0

    for i in range(3):
        ans = np.zeros((5000, 98))
        for key in params:
            ans += params[key] * f_probas[i][key]

        s += f1_score(y_folds[i], ansFromProba(ans) , average='samples')
        #print f1_score(y_folds[i], ansFromProba(ans) , average='samples')

    print 'i: {1}, mean: {0}'.format(s/ 3, counter)
    counter += 1

i: 0, mean: 0.515462539683


In [217]:
def ansFromProba2(y):
    y_copy = np.copy(y)
    ans = np.zeros(y_copy.shape)
    ans[np.arange(y.shape[0]), y_copy.argmax(1)] = 1
    
    y_copy[range(y.shape[0]), y.argmax(1)] = 0
    ans[np.arange(y.shape[0]), y_copy.argmax(1)] = 1
    
    return ans

Тут пытался увеличить вероятности некотоых тегов после предсказания первого (матрица improvments). Считал ее наивно увеличивая вероятности тем тегам, которые встречабтся с данным чаще всего. Это немного лучше, чем длины, но я почему-то это не отправил.  

In [381]:
def addSecond(ans, proba, decisioner, improvment, t):
    proba_cut = np.copy(proba)
    proba_cut[np.arange(decisioner.shape[0]), decisioner.argmax(1)] = 0
    
    #print np.sort(proba_cut, axis = None)[-100:]
    proba_cut += improvment[decisioner.argmax(1)] ** 1/4  *3
    for i in range(ans.shape[0]):
        if any(proba_cut[i] >= t):
            #print 'any!'
            ans[i][proba_cut[i] >= t] = 1
    return ans

In [449]:
s = 0
t = 0.35

for i in range(3):
    ans = np.zeros((5000, 98))
    for key in params:
        ans += best_params[key] * f_probas[i][key]

    ones_ans = ansFromProba(ans)
    s += f1_score(y_folds[i], addSecond(ansFromProba(ans), ans, ans, improvment, t) , average='samples')
    print f1_score(y_folds[i], addSecond(ansFromProba(ans), ans, ans, improvment, t) , average='samples')

print ' mean: {0}'.format(s/ 3, counter)

0.512201919192
0.527317516519
0.51471047619
 mean: 0.5180766373


In [525]:

from sklearn.linear_model import Ridge

add = 0.2
c = 1
lens_folds = []

LinReg = Ridge(alpha=c, fit_intercept = True)
reg_train = X_train_PCA[:10000]
reg_label = y_train[:10000]
LinReg.fit(reg_train, reg_label.sum(1))
    
lens_folds.append(np.round(LinReg.predict(X_train_PCA[10000:]) + add))

LinReg = Ridge(alpha=c, fit_intercept = True)
reg_train = np.vstack((X_train_PCA[:5000], X_train_PCA[10000:]))
reg_label = np.vstack((y_train[:5000], y_train[10000:]))
LinReg.fit(reg_train, reg_label.sum(1))

lens_folds.append(np.round(LinReg.predict(X_train_PCA[5000:10000]) + add))

LinReg = Ridge(alpha=c, fit_intercept = True)
reg_train = X_train_PCA[5000:]
reg_label = y_train[5000:]
LinReg.fit(reg_train, reg_label.sum(1))
    
lens_folds.append(np.round(LinReg.predict(X_train_PCA[:5000]) + add))

In [526]:
lens_folds[0].sum()

5476.0

In [527]:
s = 0
for i in range(3):
    ans = np.zeros((5000, 98))
    for key in params:
        ans += best_params[key] * f_probas[i][key] 
    
    ans[range(ans.shape[0]), ans.argmax(1)] = 100
    ans += improvment[ans.argmax(1)] ** 1/2
    s += f1_score(y_folds[i], produceOutByLen(ans, lens_folds[i].reshape(-1,1)) , average='samples')
    print f1_score(y_folds[i], produceOutByLen(ans, lens_folds[i].reshape(-1,1)), average='samples')

print ' mean: {0}'.format(s/ 3, counter)

(5000, 98)
(5000, 98)
0.500986103896
(5000, 98)
(5000, 98)
0.516225281385
(5000, 98)
(5000, 98)
0.496389047619
 mean: 0.504533477633


линейная мета модель

In [501]:
s = 0
for i in range(3):
    
    RF = RFClassifier(n_estimators=50, n_jobs=6)
    
    data = np.ones((5000, 1))
    for key in params:
        data = np.hstack((data,f_probas[i][key]))
    
    train = data[1000:]
    test = data[:1000]
    
    RF.fit(train, y_folds[i][1000:])
    
    
    s += f1_score(y_folds[i][:1000], ansFromProba(RF.predict_proba(test)) , average='samples')
    print f1_score(y_folds[i][:1000], ansFromProba(RF.predict_proba(test)), average='samples')

print ' mean: {0}'.format(s/ 3, counter)

0.4458
0.501466666667
0.515233333333
 mean: 0.4875


добавить, как фичу

In [490]:
s = 0
t = 0.35
th = 3000

aaaaa = []
rfff = []
for i in range(3):
    ans = np.zeros((5000, 98))
    for key in best_params:
        ans += best_params[key] * f_probas[i][key]

    ones_ans = ansFromProba(ans)
    
    data = np.hstack((ans, ones_ans))
    
    train = data[:th]
    test = data[th:] 
    
    train_label = y_folds[i][:th]
    test_label = y_folds[i][th:]
    
    train_label = train_label - ansFromProba(ans[:th])
    train_label[train_label < 0] = 0
    
    RF_meta = RFClassifier(n_estimators = 50, n_jobs = 6)
    RF_meta.fit(train, train_label)
    
    rf_pred = RF_meta.predict(test)
    
    aaaaa.append(ans)
    rfff.append(rf_pred)
    s += f1_score(test_label, np.round((ansFromProba(ans[th:]) + postprocessWithT(rf_pred, 0.5))/2 +0.1 ), average='samples')
    print f1_score(test_label, np.round((ansFromProba(ans[th:]) + postprocessWithT(rf_pred, 0.5))/2 + 0.1 ), average='samples')

print ' mean: {0}'.format(s/ 3, counter)

0.529076190476
0.534083333333
0.48935
 mean: 0.517503174603


In [253]:
best_params

{'MNB_proba': 9.7144514654701197e-17,
 'RF_proba': 0.64999999999999991,
 'SGD_proba': 0.050000000000000003,
 'features_proba': 0.29999999999999999}

0.515462539683

# For kaggle

In [535]:
add = 0.2
c = 1

LinReg = Ridge(alpha=c, fit_intercept = True)
reg_train = X_train_PCA
reg_label = y_train
LinReg.fit(reg_train, reg_label.sum(1))
    
lens = (np.round(LinReg.predict(X_test) + add))

In [204]:
def PrepareData(train_data, train_labels, test_data, nplexes) :
    probas = dict()
    new_features = addSimplePredFeatures(train_data, train_labels, nplexes, 5)
    new_features = np.nan_to_num(new_features)
    new_features_normed = (new_features / new_features.sum(1).reshape(-1,1))
    new_features_normed = np.nan_to_num(new_features_normed)
        
    print 'teaching RF ......'
    new_features_RFclassifier = RFClassifier(n_estimators = 1000, n_jobs = 6)
    new_features_RFclassifier.fit(new_features_normed, train_labels)
        
    print 'RF done...... \n portraits.....'
    portraits = createPortraits(nplexes, train_data, train_labels, False)
    print 'done......'
    test_features = predictSimple(test_data, portraits)
        
    test_features = np.nan_to_num(test_features)
    test_features_normed = (test_features / test_features.sum(1).reshape(-1,1))
    test_features_normed = np.nan_to_num(test_features_normed)
    
    test_features_proba = nplexProbToClassProb(test_features_normed, nplexes)
        
    RF_proba = new_features_RFclassifier.predict_proba(test_features_normed)
        
    SGD = SGDMulticlassClassifier()
        
    SGD.fit(train_data, train_labels)
    SGD_proba = SGD.predict_proba(test_data)
        
    MNB = MultinomialNB()
    train_data_split, train_label_split = splitData(train_data, train_labels)
    MNB.fit(train_data_split, train_label_split.argmax(1))
        
    MNB_labels = np.unique(sorted(train_labels.argmax(1)))
    MNB_proba = updateMNBproba(MNB_labels, np.arange(98), MNB.predict_proba(test_data))
        
    print 'All probas calculated....\n'
    probas['features_normed'] = test_features_normed
    probas['features_proba'] = test_features_proba
    probas['RF_proba'] = RF_proba
    probas['SGD_proba'] = SGD_proba
    probas['MNB_proba'] = MNB_proba
        

    return probas
        

In [206]:
kaggle_probas = PrepareData(X_train, y_train, X_test, np.array(all_nplexes)[np.array(all_nplexes).sum(1) == 1])

0.466566666667
0.471395238095
0.479895238095
0.478655555556
0.448777777778
teaching RF ......
RF done...... 
 portraits.....
done......
(15000, 98)
All probas calculated....



In [208]:
kaggle_to_save = np.zeros((X_test.shape[0], 1))
for key in kaggle_probas:
    if key != 'features_normed':
        kaggle_to_save = np.hstack((kaggle_to_save, kaggle_probas[key]))

In [210]:
pd.DataFrame(kaggle_to_save).to_csv('kaggle_all_fin.csv')

In [543]:
ans = np.zeros((X_test.shape[0], 98))

for key in best_params:
    ans += best_params[key] * kaggle_probas[key]
    

In [541]:
lens.sum()

15590.0

In [544]:
write_labels_to_file(produceOutByLen(ans, lens.reshape(-1,1)), 'rf+corrs+SGD+MNB_by_lens.csv')

(15000, 98)


Вот и все. Возможно можно было чуть получше выжать из этого сочетания моделей. 